# 🎭 GERADOR DE POEMAS COM IA E FEEDBACK

## Objetivo do Sistema

Este projeto implementa um **sistema completo de geração de poemas** usando IA generativa via API, com as seguintes funcionalidades:

- ✅ **Geração de poemas**: Uso de LLM (Large Language Model) via API
- ✅ **Feedback do usuário**: Coleta estruturada de avaliação do poema
- ✅ **Novo Chat vs Chat Contínuo**: Diferenciação entre iniciar nova conversa ou continuar com histórico
- ✅ **Interface interativa**: Widgets no Jupyter para experiência intuitiva
- ✅ **Melhoria automática**: Refinamento de prompts baseado no feedback

## Responsabilidades

- **Grupo 1**: Integração com LLM e chamadas à API
- **Grupo 2**: Interface interativa com widgets
- **Grupo 3**: Sistema de feedback e refinamento de prompts

---

## 2. IMPORTS E CONFIGURAÇÕES GERAIS

### Bibliotecas Utilizadas

Este projeto utiliza as seguintes bibliotecas Python:

- **`requests`**: Para fazer requisições HTTP à API da LLM
- **`ipywidgets`**: Para criar a interface interativa com widgets no Jupyter
- **`IPython.display`**: Para exibir conteúdo formatado (HTML, imagens, etc)
- **`json`**: Para manipular dados em formato JSON
- **`datetime`**: Para registrar timestamps das execuções
- **`os`**: Para ler variáveis de ambiente (como a chave da API)

### Variável de Ambiente

A chave da API será lida a partir da variável de ambiente `LLM_API_KEY`. Para configurar:

```bash
# No Windows (PowerShell)
$env:LLM_API_KEY = "sua_chave_aqui"

# Ou no arquivo .env
LLM_API_KEY=sua_chave_aqui
```

In [ ]:
import requests
import json
import os
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Configuração da API
API_KEY = os.getenv("LLM_API_KEY", "demo_key")  # Se não houver chave, usa "demo_key" para testes
API_URL = "https://api.openai.com/v1/chat/completions"  # Exemplo: OpenAI

print("✅ Bibliotecas importadas com sucesso!")
print(f"🔑 API_KEY configurada: {'Sim (chave do ambiente)' if os.getenv('LLM_API_KEY') else 'Não (usando modo demo)'}")

## 3. MODELO DE CONVERSAÇÃO: NOVO CHAT vs CHAT CONTÍNUO

### Conceito

Este sistema diferencia dois modos de operação:

#### **Novo Chat** 🔄
- Inicia uma **conversa completamente nova**
- O histórico de mensagens anteriores é **descartado**
- Cada poema é gerado de forma **independente**
- Útil quando: quer começar um novo tema

#### **Chat Contínuo** ➡️
- **Mantém o histórico** de mensagens anteriores
- A LLM "lembra" dos poemas e feedback anteriores
- Permite refinamento **iterativo** do resultado
- Útil quando: quer melhorar um poema com feedback

### Implementação Técnica

```
historico = [
    {"role": "user", "content": "Crie um poema sobre amor"},
    {"role": "assistant", "content": "Poema gerado..."},
    {"role": "user", "content": "Mais triste!"},
    {"role": "assistant", "content": "Poema refinado..."}
]
```

**Novo Chat**: `historico = []` (vazio)  
**Chat Contínuo**: `historico` continua crescendo com cada mensagem

## 4. FUNÇÃO GERAL DE CHAMADA DA API (Grupo 1)

### Responsabilidade

Esta função é responsabilidade do **Grupo 1** e faz:

1. **Envia o prompt** para a LLM via API
2. **Recebe a resposta** da IA
3. **Inclui o histórico** de mensagens conforme o modo (novo ou contínuo)
4. **Trata erros** e retorna o texto gerado

### Estrutura

- **Input**: `prompt` (texto do usuário), `system_message` (instrução para a IA), `historico` (opcional)
- **Output**: Texto gerado pela LLM
- **Fallback**: Se a API não funcionar, retorna exemplo pré-definido

In [ ]:
# ============================================
# GRUPO 1: INTEGRAÇÃO COM API DA LLM
# ============================================

def chamar_llm(prompt, system_message="Você é um poeta criativo.", historico=None):
    """
    Chama a LLM (Language Model) via API para gerar texto.
    
    Args:
        prompt (str): Texto do usuário/prompt
        system_message (str): Instrução do sistema para a IA
        historico (list): Histórico de mensagens anteriores (opcional)
    
    Returns:
        str: Resposta da LLM
    """
    
    # Se historico é None, cria lista vazia
    if historico is None:
        historico = []
    
    # Prepara as mensagens para o payload
    messages = historico.copy()  # Cópia do histórico
    messages.append({"role": "user", "content": prompt})
    
    # Prepara o payload
    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "system", "content": system_message}] + messages,
        "temperature": 0.8,
        "max_tokens": 300
    }
    
    try:
        # Faz a requisição à API
        headers = {"Authorization": f"Bearer {API_KEY}"}
        response = requests.post(API_URL, json=payload, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Extrai o texto da resposta
        resultado = response.json()["choices"][0]["message"]["content"]
        return resultado.strip()
    
    except Exception as e:
        # Fallback: retorna um exemplo se a API falhar
        print(f"⚠️ Erro ao chamar API: {e}")
        print("📌 Usando modo demo (exemplo pré-definido)")
        return "Nuvem branca no céu azul,\nsonho que passa e não fica,\ncoração que bate e cala,\nsilêncio que fala."

print("✅ Função chamar_llm() definida! (Grupo 1)")

## 5. GERAÇÃO DE POEMA (Grupo 1)

### Responsabilidade

Esta função também é do **Grupo 1** e:

1. **Usa a função `chamar_llm()`** para chamar a API
2. **Pede para gerar um poema** com regras específicas
3. **Define restrições**: Português, 4-12 versos, sem explicações extras
4. **Aceita o modo**: Novo chat (limpa histórico) ou continuação (mantém histórico)

### Regras para o Poema

- 🇧🇷 Língua: Obrigatoriamente **português**
- 📏 Extensão: **4 a 12 versos**
- 🎨 Formato: Apenas o poema, **sem explicações**
- 🔄 Modo: Respeita se é novo chat ou continuação

In [ ]:
def gerar_poema(prompt, historico, modo="novo_chat"):
    """
    Gera um poema baseado no prompt do usuário.
    
    Args:
        prompt (str): Descrição do poema desejado
        historico (list): Histórico de mensagens anteriores
        modo (str): "novo_chat" ou "continuacao"
    
    Returns:
        dict: {"poema": str, "historico_atualizado": list}
    """
    
    # Se é novo chat, limpa o histórico
    if modo == "novo_chat":
        historico = []
    
    # System message específico para poesia
    system_msg = """Você é um poeta criativo e experiente. 
    Seu objetivo é gerar APENAS poemas em português.
    Regras:
    - SEMPRE em português
    - 4 a 12 versos
    - SEM explicações ou comentários extras
    - Responda SOMENTE com o poema"""
    
    # Prompt preparado
    prompt_preparado = f"Crie um poema sobre: {prompt}"
    
    # Chama a LLM
    poema = chamar_llm(prompt_preparado, system_msg, historico)
    
    # Atualiza o histórico
    historico_novo = historico.copy()
    historico_novo.append({"role": "user", "content": prompt_preparado})
    historico_novo.append({"role": "assistant", "content": poema})
    
    return {
        "poema": poema,
        "historico_atualizado": historico_novo
    }

print("✅ Função gerar_poema() definida! (Grupo 1)")

## 6. ESTRATÉGIAS DE FEEDBACK E REFINAMENTO DE PROMPT (Grupo 3)

### Responsabilidade

O **Grupo 3** é responsável por:

1. **Receber feedback** do usuário sobre o poema
2. **Analisar o feedback** e extrair sugestões
3. **Combinar** poema anterior + feedback + contexto
4. **Gerar novo prompt** usando prompt engineering
5. **Diferenciar** novo chat vs continuação

### Estratégias de Refinamento

#### Reescrita Guiada
- Input: Poema original + Feedback
- Output: Novo prompt que pede à LLM refazer o poema com as sugestões

#### Análise de Sentimento
- Detecta se o feedback é positivo, negativo ou neutro
- Ajusta a abordagem do novo prompt

#### Histórico Contextual
- Se é "continuação", usa histórico para manter consistência
- Se é "novo chat", começa do zero

In [ ]:
# ============================================
# GRUPO 3: FEEDBACK E REFINAMENTO
# ============================================

def refinar_prompt(prompt_original, poema_anterior, feedback):
    """
    Refina o prompt baseado no feedback do usuário.
    
    Args:
        prompt_original (str): Prompt que gerou o poema anterior
        poema_anterior (str): Poema que foi gerado
        feedback (str): Feedback do usuário
    
    Returns:
        str: Novo prompt refinado
    """
    
    # Estratégia: Reescrita Guiada
    novo_prompt = f"""
    Poema anterior: {poema_anterior}
    
    Feedback do usuário: {feedback}
    
    Baseado no feedback acima, reescreva o poema sobre: {prompt_original}
    Incorpore as sugestões fornecidas.
    IMPORTANTE: Responda APENAS com o poema refinado, sem explicações.
    """
    
    return novo_prompt.strip()

def processar_feedback(feedback):
    """
    Processa o feedback para extrair palavras-chave.
    
    Args:
        feedback (str): Texto do feedback
    
    Returns:
        dict: Análise do feedback
    """
    
    feedback_lower = feedback.lower()
    
    # Palavras-chave para análise simples
    palavras_positivas = ["bom", "ótimo", "perfeito", "adorei", "excelente"]
    palavras_negativas = ["ruim", "chato", "fraco", "não gostei"]
    palavras_refinamento = ["mais", "menos", "triste", "alegre", "rápido", "lento"]
    
    positivo = any(palavra in feedback_lower for palavra in palavras_positivas)
    negativo = any(palavra in feedback_lower for palavra in palavras_negativas)
    tem_refinamento = any(palavra in feedback_lower for palavra in palavras_refinamento)
    
    return {
        "é_positivo": positivo,
        "é_negativo": negativo,
        "pede_refinamento": tem_refinamento,
        "sentimento": "positivo" if positivo else ("negativo" if negativo else "neutro")
    }

print("✅ Funções de feedback definidas! (Grupo 3)")

## 7. INTERFACE DO USUÁRIO (Grupo 2)

### 📱 Interface Interativa com Gradio

A **interface completa** foi implementada em um notebook separado com recursos avançados:

✨ **Funcionalidades Implementadas**:
- ✅ Campo de entrada para tema do poema
- ✅ Seleção de estilo (Romântico, Haicai, Soneto, etc)
- ✅ Configuração de tamanho e criatividade
- ✅ Botão "Gerar Poema"
- ✅ Sistema de refinamentos estruturados (Adicionar, Editar, Remover)
- ✅ Múltiplos refinamentos em sequência
- ✅ Versionamento automático de poemas
- ✅ Interface responsiva e moderna com Gradio

### 🎨 Responsabilidade do Grupo 2

O **Grupo 2** implementou:
1. Interface visual moderna com Gradio
2. Sistema de feedback estruturado
3. Fluxo de interação intuitivo
4. Validações e mensagens de erro
5. Histórico de refinamentos

### 📝 Como Usar

Execute o notebook **`interface.ipynb`** localizado na mesma pasta deste notebook.

Ou execute a célula abaixo para carregar a interface aqui mesmo.

In [ ]:
# ============================================
# EXECUTAR INTERFACE GRADIO
# ============================================

# Executar a interface Gradio do arquivo interface.ipynb
%run interface.ipynb

## 8. EXECUÇÕES COMENTADAS (OBRIGATÓRIO PARA ENTREGA)

### Instruções

Nesta seção, vocês devem **registrar 10+ execuções** do sistema. Para cada execução, documentem:

1. **Prompt inicial**: Qual foi o tema/descrição do poema solicitado
2. **Poema gerado**: O poema que a IA criou
3. **Feedback**: Que feedback foi dado (se houver)
4. **Poema refinado**: O resultado após aplicar o feedback (se houver)
5. **Análise do resultado**: Qual foi melhor? Por quê? O que funcionou?

### Template para Cada Execução

```
## Execução #X: [Tema/Descrição Breve]

**Prompt**: [Texto do prompt]

**Poema Gerado**:
[Poema aqui]

**Feedback**: [Feedback dado ou "Sem feedback"]

**Poema Refinado** (se houver):
[Poema refinado aqui ou "N/A"]

**Análise**: [Escrever o que achou, se funcionou, se precisa melhorar]

---
```

### Comece Aqui!

Use a interface acima para fazer suas execuções. Após cada uma, copie o resultado aqui embaixo com o template acima.

---

### Execução #1: Exemplo - Tema "A Lua"

**Prompt**: Uma poesia romântica sobre a lua iluminando a noite

**Poema Gerado**:
```
A lua prata brilha lá,
Derramando luz suave e doce,
Na noite escura dança e voa,
Sussurrando histórias de paz.
```

**Feedback**: Mais nostálgico e melancólico

**Poema Refinado**:
```
A lua pálida desce lentamente,
Saudade em cada raio de prata,
Noites que passam e nunca voltam,
Coração que chora sem som.
```

**Análise**: O refinamento funcionou bem! O feedback "mais nostálgico" resultou em um poema melancólico. As palavras "saudade", "nunca voltam" e "coração que chora" transmitem a emoção desejada.

---

### Execução #2: [Faça a sua!]

**Prompt**: 

**Poema Gerado**:

**Feedback**: 

**Poema Refinado**:

**Análise**: 

---

## 9. NOTAS PARA O RELATÓRIO

### Este é um espaço reservado para anotar informações importantes para o relatório final!

#### 📝 Dificuldades Encontradas

- [Grupo 1 - IA/API]: Escreva aqui as dificuldades...
- [Grupo 2 - Interface]: Escreva aqui as dificuldades...
- [Grupo 3 - Feedback]: Escreva aqui as dificuldades...

#### 🤖 Escolhas do Modelo de IA

- **API escolhida**: [OpenAI / HuggingFace / Cohere / Outra]
- **Modelo específico**: [gpt-3.5-turbo / distilgpt2 / command / outro]
- **Justificativa**: Por que escolheram este modelo?
- **Configurações**: Temperatura, max_tokens, etc.

#### 🧠 Uso de IA Generativa no Desenvolvimento

Registrem aqui como usaram IA generativa para ajudar no desenvolvimento:

- [ ] ChatGPT para debugar erros
- [ ] Copilot para sugerir código
- [ ] Claude para explicar conceitos
- [ ] Outro: ________________

**Descrição**: Como foi usado? Qual foi o resultado?

#### 💡 Aprendizados dos Integrantes

**[Integrante 1]**: (seu aprendizado aqui)
- O que aprendeu sobre programação?
- O que aprendeu sobre IA?
- O que foi desafiador?

**[Integrante 2]**: (seu aprendizado aqui)

**[Integrante 3]**: (seu aprendizado aqui)

#### 🎯 Conclusões e Reflexões

- Como o sistema funcionou no geral?
- O que funcionou bem?
- O que poderia melhorar?
- Qual foi a contribuição de cada grupo?
- Se tivessem mais tempo, o que fariam?

---

**✅ Notebook completo e pronto para uso!**